Set up the synspy and plotting environment

In [88]:
from deriva.core import HatracStore, ErmrestCatalog, get_credential
from synspy.analyze.pair import SynapticPairStudy

import synapse_utils

import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

Make connection to the synapse server.  You need to make sure you have logged in using the DERIVA login agent first

In [89]:
credential = get_credential("synapse.isrd.isi.edu")
ermrest_catalog = ErmrestCatalog('https', 'synapse.isrd.isi.edu', 1, credential)
hatrac_store = HatracStore('https', 'synapse.isrd.isi.edu', credentials=credential)

In [92]:
study_entities = synapse_utils.get_synapse_studies()
study_uri = study_entities.uri

print('Identified %d studies' % len(study_entities))

Identified 64 studies


In [97]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',  
    'PrcDsy20170613A': 'conditioned', 
    'PrcDsy20170615A': 'unconditioned', 
    'PrcDsy20170613B': 'control',
    'PrcDsy20171030A': 'unknown',
    'PrcDsy20171030B': 'unknown'
}

for i in study_entities:
    if protocol_types[i['Protocol']] == 'aversion':
        i['Type'] = 'learner' if i['Learner'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['Protocol']]
    study_list.append(i)

{'AfterURL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s2-registered.csv:K2FFS4UIS422GQF5EXYWA5372A',
 'BeforeURL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s1-registered.csv:YND3C4Q2LAEPNNNAGTNTNK36RE',
 'Learner': True,
 'Protocol': 'PrcDsy20160101A',
 'Study': 'SynStd6025',
 'Subject': 'ZfDsy20160116A',
 'Type': 'learner'}

Pick a study that we want to look at and set the parameters for the sweep.

In [71]:
syn_study_id = 'SynStd8175' # the study we'll analyze
syn_pair_radii = (2.0, 3.0, 4.0) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

In [72]:
study = SynapticPairStudy.from_study_id(ermrest_catalog, syn_study_id)
study.retrieve_data(hatrac_store)

s1_to_s2, s2_to_s1 = study.syn_pairing_maps(syn_pair_radii, syn_dx_core_ratio, syn_core_max_ratio)

In [79]:
# get results for different radii

unpaired_before, unpaired_after, paired_before, paired_after = {}, {}, {}, {}

# Create a list of pandas, one for each radii
for i,r in enumerate(syn_pair_radii):
    unpaired1 = study.get_unpaired(s1_to_s2[i,:], study.s1)
    unpaired2 = study.get_unpaired(s2_to_s1[i,:], study.s2)
    paired1, paired2 = study.get_pairs(s1_to_s2[i,:], study.s1, study.s2)
    
    unpaired_before[r] = pd.DataFrame(unpaired1[:,0:5], columns=['x','y','z','core','hollow'])
    unpaired_after[r] = pd.DataFrame(unpaired2[:,0:5], columns=['x','y','z','core','hollow'])
    paired_before[r]  = pd.DataFrame(paired1[:,0:5], columns=['x','y','z','core','hollow'])
    paired_after[r] = pd.DataFrame(paired2[:,0:5], columns=['x','y','z','core','hollow'])

Basic Statistics

In [80]:
for r in  syn_pair_radii:
    print('Radius {0}'.format(r))
    print('    unpaired before {0}'.format(len(unpaired_before[r].index)))
    print('    unpaired after {0}'.format(len(unpaired_after[r].index)))
    print('    paired {0}'.format(len(paired_before[r].index)))

Radius 2.0
    unpaired before 2774
    unpaired after 3098
    paired 732
Radius 3.0
    unpaired before 1634
    unpaired after 1958
    paired 1872
Radius 4.0
    unpaired before 1034
    unpaired after 1358
    paired 2472


Plot histograms of different radii

In [81]:
for r in  syn_pair_radii:
    before = paired_before[r]
    after = paired_after[r]
    pair_distance = np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)
                
    pair_histo = go.Histogram(x=pair_distance,)

    hist_layout = go.Layout(
        title='Pair Histogram Radius {0}'.format(r),
        xaxis=dict(
            title='Distance'
        ),
        yaxis=dict(
            title='Count'
        ),
        bargap=0.2,
        bargroupgap=0.1
    )

    hist_fig = go.Figure(data=[pair_histo], layout=hist_layout)

    py.offline.iplot(hist_fig, filename='pair-distance')

In [82]:
for r in  syn_pair_radii:
    before = paired_before[r]
    after = paired_after[r]
  
    before_synapses = go.Scatter3d(
        x=before['x'],
        y=before['y'],
        z=before['z'],
        name='Before Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    after_synapses = go.Scatter3d(
        x=after['x'],
        y=after['y'],
        z=after['z'],
        name='After Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [ before_synapses, after_synapses]

    layout = go.Layout(
         title='Paired Synapapses at Radius {0}'.format(r),
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

Now look at the unpaired synapses

In [83]:
for r in  syn_pair_radii:
    before = unpaired_before[r]
    after = unpaired_after[r]
    
    before_synapses = go.Scatter3d(
        x=before['x'],
        y=before['y'],
        z=before['z'],
        name='Before Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    after_synapses = go.Scatter3d(
        x=after['x'],
        y=after['y'],
        z=after['z'],
        name='After Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.5
        )
    )

    data = [before_synapses, after_synapses]

    layout = go.Layout(
        title='Unpaired Synapapses at Radius {0}'.format(r),
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)